## ISyE6669 HW10 - Graham Billey, Spring 2020

#### 1. Column Generation

> In this problem, we want to walk you through *one* iteration of the column generation in solving the cutting stock problem.  Consider the following formulation
>
> $$ \min \sum_{j=1}^{n} x_j $$
> 
> $$ s.t.\ \sum_{j=1}^{N} A_j x_j\ =\ \vec b $$
> 
> $$ x_j\ \geq 0,\ \forall\ j\ =\ 1,...,N. $$
> 
> The problem has the following data. Customers need three types of smaller widths: $ w_1 = 5, w_2 = 12, w_3 = 16 $ with quantities $ b_1 = 150, b_2 = 100, b_3 = 80 $. The width of a big roll is $ W = 200 $.
> 
> 1) Assume the column generation algorithm starts from the following initial patterns:
> 
> $$ A_1 = \begin{bmatrix} 40 \\ 0 \\ 0 \end{bmatrix}, A_2 = \begin{bmatrix} 0 \\ 16 \\ 0 \end{bmatrix}, A_3 = \begin{bmatrix} 0 \\ 0 \\ 12 \end{bmatrix}. $$
> 
> Write down the restricted master problem (RMP) using these patterns.

The RMP is, in general:

$$ \min \sum_{j \in I} x_j $$

$$ s.t.\ \sum_{j \in I} A_j x_j\ =\ \vec b $$

$$ x_j\ \geq 0,\ \forall\ j\ \in I $$

where...

$$ I = [1,2,3], \quad \vec b = \begin{bmatrix} 150 \\ 100 \\ 80 \end{bmatrix} $$

Explicitly, the RMP at the beginning stage of the cutting stock problem is:

$$ \min \quad x_1 + x_2 + x_3 $$

$$ s.t.\ \begin{bmatrix} 40 \\ 0 \\ 0 \end{bmatrix} x_1 + \begin{bmatrix} 0 \\ 16 \\ 0 \end{bmatrix} x_2 + \begin{bmatrix} 0 \\ 0 \\ 12 \end{bmatrix} x_3 =\ \begin{bmatrix} 150 \\ 100 \\ 80 \end{bmatrix} $$

$$ x_1, x_2, x_3\ \geq 0 $$

> 2) Solve RMP in CVX (Python or Matlab).  Write down the optimal solution, the optimal basis $ B $, and its inverse $ B^{−1} $.  Find the optimal dual solution $ \hat{y} \geq c_B^{\top} B^{−1} $.  To take the inverse of $ B $, you can use your favorite calculator or computer program.  In this iteration, you could solve this LP by hand.  But we ask you to set up the code in CVX and solve it using CVX. This code will be used in later iterations.

In [1]:
import numpy as np
import cvxpy as cp

In [24]:
# question1_RMP_step1

x = cp.Variable(3)

A = np.array([[40, 0,  0],
              [0, 16, 0],
              [0, 0, 12]])

b = np.array([150, 100, 80])

#Objective function
obj = cp.Minimize(sum(x))
prob = cp.Problem(obj, [A @ x >= b,
                        x >= 0])
prob.solve(solver='GLPK_MI')

print(f'The optimal value is: {round(prob.value,2)}')
print(f'The optimal x is: {np.round(x.value,2)} \n')

###### Print answers to questions below
print('----- question1_RMP_step1 ----- \n')

print(f'The Basis is : \n {A} \n')

B_inv = np.linalg.inv(A) 
print(f'The B_inv is : \n {np.round(B_inv,3)} \n')

y_hat = np.ones(3).T@B_inv
print(f'The optimal dual solution is : \n {np.round(y_hat,3)} \n')

The optimal value is: 16.67
The optimal x is: [3.75 6.25 6.67] 

----- question1_RMP_step1 ----- 

The Basis is : 
 [[40  0  0]
 [ 0 16  0]
 [ 0  0 12]] 

The B_inv is : 
 [[0.025 0.    0.   ]
 [0.    0.062 0.   ]
 [0.    0.    0.083]] 

The optimal dual solution is : 
 [0.025 0.062 0.083] 



> 3) Write down  the  pricing  problem, i.e. the knapsack problem using the above data and the optimal dual solution you found.

The knapsack problem is, in general:

$$ \max \sum_{j} a_j $$

$$ s.t. \sum_{j} w_j a_j \leq W $$

$$ a_j \geq 0,\ \forall j $$

Explicitly, the knapsack problem at this stage of the cutting stock problem is:

$$ \max a_1 + a_2 + a_3 $$

$$ s.t. 5 a_1 + 12 a_2 + 16 a_3 \leq 200 $$

$$ a_1, a_2, a_3 \geq 0 $$

> 4) Solve  the  pricing  problem  in  CVX.  Should  we  terminate  the  column  generation  algorithm at this point?  Explain.  If the column generation should continue, what is the new pattern generated by the pricing problem?

In [25]:
# question1_Pricing_step1

a = cp.Variable(3, integer=True)
w = np.array([5, 12, 16])
W = 200

#Objective function
obj = cp.Maximize(y_hat@a)
prob = cp.Problem(obj, [w@a <= W,
                        a>=0])
prob.solve(verbose=False, solver='GLPK_MI')

print(f'The optimal value is: {np.round(prob.value,3)}')
print(f'The optimal a is: {np.round(a.value)} \n')

###### Print answers to questions below
print('----- question1_Pricing_step1 ----- \n')

mrc = 1-prob.value
print(f'The minimum reduced cost is : \n {np.round(mrc,3)} \n')

if mrc < 0:
    print('Since the minimum reduced cost is < 0, the current solution is not optimal. \n')
    print(f'The new pattern generated by the pricing problem is {np.round(a.value)}^T. \n')
else:
    print('The current solution is optimal! We are done! Woo Hoo :)')
    

The optimal value is: 1.042
The optimal a is: [ 0.  2. 11.] 

----- question1_Pricing_step1 ----- 

The minimum reduced cost is : 
 -0.042 

Since the minimum reduced cost is < 0, the current solution is not optimal. 

The new pattern generated by the pricing problem is [ 0.  2. 11.]^T. 



> 5) If the column generation should continue, then augment (RMP) with the new column and solve it in CVX again.  You can easily modify your CVX code by incorporating the new column. Write down the optimal solution, the optimal basis $ B $, and the inverse $ B^{−1} $. Compute the dual variable. Then solve the pricing problem again by modifying the date in your code. Should you terminate the column generation at this iteration?  Explain.  If the column generation should continue, do the same for all the following iterations until the column generation terminates.

In [27]:
# question1_RMP_step2

x = cp.Variable(4)

A = np.array([[40,  0,  0,  0],
              [ 0, 16,  0, 2],
              [ 0,  0, 12,  11]])

b = np.array([150, 100, 80])

#Objective function
obj = cp.Minimize(sum(x))
prob = cp.Problem(obj, [A @ x == b,
                        x >= 0])
prob.solve(solver='GLPK_MI')

print(f'The optimal value is: {round(prob.value,2)}')
print(f'The optimal x is: {np.round(x.value,2)} \n')

###### Print answers to questions below
print('----- question1_RMP_step2 ----- \n')

B = A[:,[0,1,3]]
print(f'The Basis is : \n {B} \n')

B_inv = np.linalg.inv(B) 
print(f'The B_inv is : \n {np.round(B_inv,3)} \n')

y_hat = np.ones(3).T@B_inv
print(f'The optimal dual solution is : \n {np.round(y_hat,3)} \n')

The optimal value is: 16.36
The optimal x is: [3.75 5.34 0.   7.27] 

----- question1_RMP_step2 ----- 

The Basis is : 
 [[40  0  0]
 [ 0 16  2]
 [ 0  0 11]] 

The B_inv is : 
 [[ 0.025  0.     0.   ]
 [ 0.     0.062 -0.011]
 [ 0.     0.     0.091]] 

The optimal dual solution is : 
 [0.025 0.062 0.08 ] 



In [28]:
# question1_Pricing_step2

a = cp.Variable(3, integer=True)
w = np.array([5, 12, 16])
W = 200

#Objective function
obj = cp.Maximize(y_hat@a)
prob = cp.Problem(obj, [w@a <= W,
                        a>=0])
prob.solve(verbose=False, solver='GLPK_MI')

print(f'The optimal value is: {np.round(prob.value,3)}')
print(f'The optimal a is: {np.round(a.value)} \n')

###### Print answers to questions below
print('----- question1_Pricing_step2 ----- \n')

mrc = 1-prob.value
print(f'The minimum reduced cost is : \n {np.round(mrc,3)} \n')

if mrc < 0:
    print('Since the minimum reduced cost is < 0, the current solution is not optimal. \n')
    print(f'The new pattern generated by the pricing problem is {np.round(a.value)}^T. \n')
else:
    print('The current solution is optimal! We are done! Woo Hoo :)')

The optimal value is: 1.038
The optimal a is: [ 4. 15.  0.] 

----- question1_Pricing_step2 ----- 

The minimum reduced cost is : 
 -0.038 

Since the minimum reduced cost is < 0, the current solution is not optimal. 

The new pattern generated by the pricing problem is [ 4. 15.  0.]^T. 



In [30]:
# question1_RMP_step3

x = cp.Variable(5)

A = np.array([[40,  0,  0,  0,  4],
              [ 0, 16,  0,  2, 15],
              [ 0,  0, 12, 11,  0]])

b = np.array([150, 100, 80])

#Objective function
obj = cp.Minimize(sum(x))
prob = cp.Problem(obj, [A @ x == b,
                        x >= 0])
prob.solve()

print(f'The optimal value is: {round(prob.value,2)}')
print(f'The optimal x is: {np.round(x.value,2)} \n')

###### Print answers to questions below
print('----- question1_RMP_step3 ----- \n')

B = A[:,[0,3,4]]
print(f'The Basis is : \n {B} \n')

B_inv = np.linalg.inv(B) 
print(f'The B_inv is : \n {np.round(B_inv,3)} \n')

y_hat = np.ones(3).T@B_inv
print(f'The optimal dual solution is : \n {np.round(y_hat,3)} \n')


The optimal value is: 16.15
The optimal x is: [3.18 0.   0.   7.27 5.7 ] 

----- question1_RMP_step3 ----- 

The Basis is : 
 [[40  0  4]
 [ 0  2 15]
 [ 0 11  0]] 

The B_inv is : 
 [[ 0.025 -0.007  0.001]
 [ 0.     0.     0.091]
 [ 0.     0.067 -0.012]] 

The optimal dual solution is : 
 [0.025 0.06  0.08 ] 



In [34]:
# question1_Pricing_step3

a = cp.Variable(3, integer=True)
w = np.array([5, 12, 16])
W = 200

#Objective function
obj = cp.Maximize(y_hat@a)
prob = cp.Problem(obj, [w@a <= W,
                        a>=0])
prob.solve(verbose=False, solver='GLPK_MI')

print(f'The optimal value is: {np.round(prob.value,3)}')
print(f'The optimal a is: {np.round(a.value)} \n')

###### Print answers to questions below
print('----- question1_Pricing_step3 ----- \n')

mrc = 1-prob.value
print(f'The minimum reduced cost is : \n {np.round(mrc,3)} \n')

if mrc < 0:
    print('Since the minimum reduced cost is < 0, the current solution is not optimal. \n')
    print(f'The new pattern generated by the pricing problem is {np.round(a.value)}^T. \n')
else:
    print('The current solution is optimal! We are done! Woo Hoo :)')

The optimal value is: 1.0
The optimal a is: [40.  0.  0.] 

----- question1_Pricing_step3 ----- 

The minimum reduced cost is : 
 0.0 

The current solution is optimal! We are done! Woo Hoo :)


> 6) Write down the final optimal solution, the optimal basis, and the optimal objective value.

In [37]:
print(f'The optimal solution is: {np.round(x.value,2)} \n')

print(f'The Basis is : \n {B} \n')

print(f'The optimal value is: 16.15')

The optimal solution is: [3.18 0.   0.   7.27 5.7 ] 

The Basis is : 
 [[40  0  4]
 [ 0  2 15]
 [ 0 11  0]] 

The optimal value is: 16.15


We will cut `3.18` patterns of $ \begin{bmatrix} 40 \\ 0 \\ 0 \end{bmatrix} $, 
            `7.27` patterns of $ \begin{bmatrix} 0 \\ 2 \\ 15 \end{bmatrix} $, and 
            `5.7`  patterns of $ \begin{bmatrix} 4 \\ 15 \\ 0 \end{bmatrix} $.
            
The total number of big rolls cut is 16.15, but in reality we cannot combine fractional rolls into one.

Therefore, the actual number of big rolls cut is `4 + 8 + 6 = 18` rolls.

### YOUR WORK HERE

> 7. For this problem, you need to submit all your codes for all the steps separately.  Name them as question1_RMP_step1, question1_Pricing_step1, question1_RMP_step2, and so on.
> 
> NOTE: Per Will Lassiter (TA), keeping all code chunks in this workbook is fine: https://piazza.com/class/k3pzx6pzx9h6t5?cid=253_f1